# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234036613010                   -0.50    7.0         
  2   -7.249814027105       -1.80       -1.40    1.0   8.09ms
  3   -7.251149789017       -2.87       -2.03    2.0   9.46ms
  4   -7.251258825106       -3.96       -2.16    3.0   12.2ms
  5   -7.251313914548       -4.26       -2.46    1.0   8.73ms
  6   -7.251337216191       -4.63       -3.01    1.0   8.74ms
  7   -7.251338720167       -5.82       -3.67    1.0   8.57ms
  8   -7.251338785818       -7.18       -3.98    2.0   10.5ms
  9   -7.251338797716       -7.92       -4.67    1.0   8.80ms
 10   -7.251338798533       -9.09       -4.93    2.0   10.2ms
 11   -7.251338798680       -9.83       -5.58    1.0   9.16ms
 12   -7.251338798703      -10.64       -5.95    3.0   12.6ms
 13   -7.251338798704      -11.97       -6.37    1.0   9.00ms
 14   -7.251338798705      -12.80       -6.87    2.0   10.5ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04815990562502615
[ Info: Arnoldi iteration step 2: normres = 0.6270711013079217
[ Info: Arnoldi iteration step 3: normres = 0.775993452820397
[ Info: Arnoldi iteration step 4: normres = 0.20405699315924394
[ Info: Arnoldi iteration step 5: normres = 0.5632766800141137
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.62e-02, 8.31e-02, 5.32e-01, 1.65e-01, 1.38e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2557842282394802
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.41e-03, 1.15e-01, 1.83e-01, 1.05e-01, 3.03e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10577447541118046
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.79e-04, 1.30e-02, 1.65e-02, 8.06e-02, 5.69e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11681601012298723
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.00e-0